In [7]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import cv2 as cv 
from os.path import join
import progressbar 
%matplotlib qt

In [10]:
#path of the images
path = r'D:\Dalhousie\datasets'
#folder = r'05 2023.07.30 NoFiducials\cropped'
folder = r'06 2023.07.30 WithFiducials\cropped'
saving_folder = r'06 2023.07.30 WithFiducials\LogGabor'

in_path=join(path,folder)
entries = os.listdir(in_path)
image_path = join(in_path,entries[333]) #Selected image
savedimage_path = join(path,saving_folder)

In [11]:
#Reading and showing image to ensure that the slected image exists in the path
img=cv.imread(image_path,0)
cv.imshow('Imagen original',img)

cv.waitKey(0)
img.shape

(666, 431)

In [12]:
#Function to create a frequency grid to create the filter.
def Grid2D(imag_shape):
    ysize,xsize = imag_shape #The grid needs to be the same size as the image
    ymid = ysize // 2
    xmid = xsize // 2
    
    if ysize % 2 == 0: #Condition to led with images with odd or even size.
        ymax = ymid - 1
    else:
        ymax = ymid

    if xsize % 2 == 0:
        xmax = xmid - 1
    else:
        xmax = xmid
    
    X,Y= np.meshgrid(np.arange(-xmid, xmax + 1),np.arange(-ymid, ymax + 1)) #Spatial domain of the grid
    X = np.fft.ifftshift(X)/xsize #Frecuency domain of the grid (also normalized)
    Y = np.fft.ifftshift(Y)/ysize
    
    return X,Y

In [13]:
def RiezFilter(img_shape): 
    u,v = Grid2D(img_shape)
    w = np.sqrt(u**2+v**2)
    w[0,0]=0.0001
    
    RiezFilt = (1j * v - u)/ w
    
    return RiezFilt

def LogGaborFilter(img_shape,scaling_factor,wavelenght,k_w0):
    
    #Create a Frequency Grid
    X,Y = Grid2D(img_shape)
    
    FilterBank = np.zeros((len(wavelenght),img_shape[0],img_shape[1]))
    
    #Determine the spatial regions to use 
    w = np.sqrt(Y**2+X**2)
    w[0,0]=0.0001 #Avoids log(0)
    phi = np.arctan2(Y,X)
    
    #User defined parameter
    m = scaling_factor
    
    for i in range(len(wavelenght)):
        
        lamb = wavelenght[i]
        w_0 = 1/(lamb)

        GabFilter = np.exp( -((np.log(w/w_0))**2 / (2*(np.log(k_w0))**2)))#+((phi-phi_0)**2/(2*sig_phi**2)) 
        GabFilter[0,0] = 0

        if (img_shape[0]%2 == 0):
            GabFilter[img_shape[0]//2,:]=0

        if (img_shape[1]%2 == 0):
            GabFilter[:,img_shape[1]//2]=0
        
        FilterBank[i,:,:] = GabFilter
    
    return FilterBank

def MonogenicSignal(image,LogGaborFilt,ReiszFilt):
    F = np.fft.fft2(image)
    
    Ffilt = np.multiply(F,LogGaborFilt)
    
    Fm1 = np.real(np.fft.ifft2(Ffilt))
    
    Fmodd = np.fft.ifft2(np.multiply(Ffilt,ReiszFilt))
    Fm2 = np.real(Fmodd)
    Fm3 = np.imag(Fmodd)
    
    return Fm1,Fm2,Fm3

def FeatureSymetry(m1,m2,m3,T):
    epsilon = 0.001
    
    odd = np.sqrt(m2**2 + m3**2)
    even = np.abs(m1)
    
    denominator = np.sqrt(even**2+odd**2)+epsilon
    
    FS_num = np.clip((even - odd - T), 0,None)
    FA_num = np.clip((odd - even - T), 0,None)
    
    FS = FS_num/denominator
    FA = FA_num/denominator
    
    FS = np.mean(FS, axis=0)
    FA = np.mean(FA, axis=0)
    
    
    return FS,FA


RF=RiezFilter(img.shape)

wl = 30*1.5**np.arange(5)
print(wl)

lG = LogGaborFilter(img.shape,1.5,wl,0.75)


m1,m2,m3 = MonogenicSignal(img,lG,RF)
FS,FA = FeatureSymetry(m1,m2,m3,0.18)

FS = (FS-FS.min())*(255/(FS.max()-FS.min()))
FS = FS.astype(np.uint8)
print(FS.max())
plt.figure()
plt.subplot(131),plt.imshow(img, cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(FS, cmap = 'gray')
plt.title('Symmetry'), plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(FA, cmap = 'gray')
plt.title('Asymmetry'), plt.xticks([]), plt.yticks([])
plt.show()


[ 30.     45.     67.5   101.25  151.875]
254


In [18]:
maximumvalue = len(entries)-3
bar = progressbar.ProgressBar(maxval=maximumvalue).start()



for ith in range (4,maximumvalue):
    
    in_image_path = join(in_path,entries[ith])
    out_image_path = join(savedimage_path,entries[ith])

    img = cv.imread(in_image_path,0)
    mask = img.copy()*0
    
    mask[img==0] = False
    mask[img>0] = True
    
    mask = mask*255
    
    RF=RiezFilter(img.shape)

    wl = np.array([70,90,110])

    lG = LogGaborFilter(img.shape,1.5,wl,0.4)
    m1,m2,m3 = MonogenicSignal(img,lG,RF)

    LocalPhase = np.arctan2(np.sqrt(m2**2+m3**2),m1)
    #LocalPhase = LocalPhase[1,:,:]
    FS,FA = FeatureSymetry(m1,m2,m3,0.18)
    
    FA = (FA-FA.min())*(255/(FA.max()-FA.min()))
    FA = FA.astype(np.uint8)
    FS = (FS-FS.min())*(255/(FS.max()-FS.min()))
    FS = FS.astype(np.uint8)
    
    
    FA[mask==0] = 255
    FS[mask==0] = 0 
    
    cv.imshow('',np.c_[img,FS,~FA])
    cv.waitKey(1)
    #cv.imwrite(out_image_path,np.c_[img,FS])
    bar.update(ith)

 99% (480 of 481) |##################### | Elapsed Time: 0:02:32 ETA:   0:00:00